In [ ]:
# ====================================
# Notebook 5.1: ALS based on Binary / Quantity Matrix
# Description:
# This notebook trains two ALS models based on binary and quantity user-item matrices.
# The models are evaluated using nDCG@20 and Recall@20.
# ====================================

In [ ]:
import os

# === Clone GitHub repository ===
repo_dir = "My-BS-Thesis"

if os.path.exists(repo_dir):
    print(f"{repo_dir} already exists. Removing it...\n")
    !rm -r {repo_dir}

!git clone https://github.com/Goshmar/My-BS-Thesis

Cloning into 'My-BS-Thesis'...
remote: Enumerating objects: 131, done.
remote: Counting objects: 100% (131/131), done.
remote: Compressing objects: 100% (126/126), done.
remote: Total 131 (delta 42), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (131/131), 201.78 MiB | 22.87 MiB/s, done.
Resolving deltas: 100% (42/42), done.


In [ ]:
# === Install dependencies from requirements.txt ===
!pip install -r My-BS-Thesis/requirements.txt -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 64.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.7/557.7 MB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.2/160.2 kB 109.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.3/27.3 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 768.5/768.5 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install implicit -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 53.4 MB/s eta 0:00:00


In [ ]:
import json
import random
import pickle
import zipfile
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.sparse import load_npz, csr_matrix
from implicit.als import AlternatingLeastSquares

In [ ]:
# === Set paths ===
processed_data = os.path.join(repo_dir, 'data', 'processed')
artifacts_dir = os.path.join(repo_dir, 'artifacts')

binary_matrix_path = os.path.join(processed_data, 'user_item_binary_matrix.npz')
quantity_matrix_path = os.path.join(processed_data, 'user_item_quantity_matrix.npz')
train_zip_path = os.path.join(processed_data, 'train_df.zip')
test_zip_path = os.path.join(processed_data, 'test_df.zip')
map_path = os.path.join(artifacts_dir, 'mapping.pkl')

In [ ]:
# === Load data ===
user_item_binary_matrix = load_npz(binary_matrix_path)
user_item_quantity_matrix = load_npz(quantity_matrix_path)
print("✅ User-item matrices loaded.")

with zipfile.ZipFile(train_zip_path, 'r') as zip_ref:
    with zip_ref.open("train_df.csv") as f:
        train_df = pd.read_csv(f)

with zipfile.ZipFile(test_zip_path, 'r') as zip_ref:
    with zip_ref.open("test_df.csv") as f:
        test_df = pd.read_csv(f)
print("✅ Train and test data loaded.")

with open(map_path, 'rb') as f:
    mappings = pickle.load(f)
user_map = mappings['user_map']
item_map = mappings['item_map']
reverse_item_map = mappings['reverse_item_map']

✅ User-item matrices loaded.
✅ Train and test data loaded.


In [ ]:
# === Train ALS on BINARY matrix ===
print("\n🔧 Training ALS on BINARY matrix...")
als_binary = AlternatingLeastSquares(factors=16, iterations=8, regularization=0.0)
als_binary.fit(user_item_binary_matrix)
print("✅ ALS Binary model trained.")


🔧 Training ALS on BINARY matrix...


/usr/local/lib/python3.11/dist-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 2 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/8 [00:00<?, ?it/s]

✅ ALS Binary model trained.


In [ ]:
# === Recommended functions ===
def make_coo_row(products):
    row_data, row_cols = [], []
    for item, quantity in products.items():
        if item in item_map:
            q = quantity[0] if isinstance(quantity, list) else quantity
            row_cols.append(item_map[item])
            row_data.append(float(q))
    return csr_matrix((row_data, ([0]*len(row_data), row_cols)), shape=(1, len(item_map)))

def recommend_items(model, products, n=30):
    row = make_coo_row(products).tocsr()
    recs = model.recommend(
        userid=0,
        user_items=row,
        N=n,
        filter_already_liked_items=True,
        recalculate_user=True
    )
    return [reverse_item_map[idx] for idx in recs[0]]

In [ ]:
def ndcg_at_k(actual, predicted, k=20):
    dcg = 0.0
    for i, p in enumerate(predicted[:k]):
        if p in actual:
            dcg += 1 / np.log2(i + 2)
    ideal_dcg = sum(1 / np.log2(i + 2) for i in range(min(len(actual), k)))
    return dcg / ideal_dcg if ideal_dcg > 0 else 0.0

def mean_ndcg_at_k(actual_list, predicted_list, k=20):
    return np.mean([ndcg_at_k(a, p, k) for a, p in zip(actual_list, predicted_list)])

def recall_at_k(actual, predicted, k=20):
    return len(set(predicted[:k]) & set(actual)) / len(actual) if actual else 0.0

def mean_recall_at_k(actual_list, predicted_list, k=20):
    return np.mean([recall_at_k(a, p, k) for a, p in zip(actual_list, predicted_list)])

In [ ]:
# === Evaluate on test set ===
actual_orders = []
predicted_orders = []

for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc="📊 Evaluating ALS Binary model"):
    try:
        products_dict = eval(row["products"])
        product_ids = list(products_dict.keys())
    except:
        continue

    if len(product_ids) < 2:
        continue

    basket = product_ids[:len(product_ids)//2]
    target = product_ids[len(product_ids)//2:]

    recommended = recommend_items(als_binary, {item: products_dict[item] for item in basket})
    actual_orders.append(target)
    predicted_orders.append(recommended)

📊 Evaluating ALS Binary model: 100%|██████████| 80000/80000 [00:51<00:00, 1561.53it/s]


In [ ]:
# === Final metrics ===
ndcg_20_binary = mean_ndcg_at_k(actual_orders, predicted_orders)
recall_20_binary = mean_recall_at_k(actual_orders, predicted_orders)

print(f"\n📈 ALS (BINARY) nDCG@20: {ndcg_20_binary:.4f}")
print(f"📈 ALS (BINARY) Recall@20: {recall_20_binary:.4f}")


📈 ALS (BINARY) nDCG@20: 0.1225
📈 ALS (BINARY) Recall@20: 0.2021


In [ ]:
# === Train ALS on QUANTITY matrix ===
print("\n🔧 Training ALS on QUANTITY matrix...")
als_quantity = AlternatingLeastSquares(factors=16, iterations=8, regularization=0.0)
als_quantity.fit(user_item_quantity_matrix)
print("✅ ALS Quantity model trained.")


🔧 Training ALS on QUANTITY matrix...


  0%|          | 0/8 [00:00<?, ?it/s]

✅ ALS Quantity model trained.


In [ ]:
# === Evaluate on test set ===
actual_orders = []
predicted_orders = []

for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc="📊 Evaluating ALS Quantity model"):
    try:
        products_dict = eval(row["products"])
        product_ids = list(products_dict.keys())
    except:
        continue

    if len(product_ids) < 2:
        continue

    basket = product_ids[:len(product_ids)//2]
    target = product_ids[len(product_ids)//2:]

    recommended = recommend_items(als_quantity, {item: products_dict[item] for item in basket})
    actual_orders.append(target)
    predicted_orders.append(recommended)

📊 Evaluating ALS Quantity model: 100%|██████████| 80000/80000 [00:46<00:00, 1724.02it/s]


In [ ]:
# === Final metrics ===
ndcg_20_quantity = mean_ndcg_at_k(actual_orders, predicted_orders)
recall_20_quantity = mean_recall_at_k(actual_orders, predicted_orders)

print(f"\n📈 ALS (QUANTITY) nDCG@20: {ndcg_20_quantity:.4f}")
print(f"📈 ALS (QUANTITY) Recall@20: {recall_20_quantity:.4f}")


📈 ALS (QUANTITY) nDCG@20: 0.0877
📈 ALS (QUANTITY) Recall@20: 0.1442
